In [ ]:
import faiss
from langchain_ollama import ChatOllama,OllamaEmbeddings
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
import faiss
import xml.etree.ElementTree as ET
import pyttsx3
from langchain_core.output_parsers import PydanticOutputParser,StrOutputParser
from typing import List
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig
import re
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
import numpy as np
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate
from langchain_core.prompts import PromptTemplate
import spacy
import pickle
import requests
from scispacy.linking import EntityLinker
import scispacy
import re
from langchain.messages import SystemMessage,HumanMessage,AIMessage,ToolMessage
from langchain.tools import tool
from langchain_community.docstore.in_memory import InMemoryDocstore
import pyobo
import time
from collections import defaultdict


USER_AGENT environment variable not set, consider setting it to identify your requests.
/home/bionik/miniconda3/envs/LLM/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#linker = pyobo.get_scispacy_entity_linker("uniprot", filter_for_definitions=False, resolve_abbreviations=True)
nlp = spacy.load('en_ner_jnlpba_md')
nlp.disable_pipes("tagger", "parser")
nlp.add_pipe("sentencizer")
linker = pyobo.get_scispacy_entity_linker("hgnc", filter_for_definitions=False, resolve_abbreviations=True,)


In [5]:
def fetch_ncbi_data(query):
    base_url_esearch='https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    params_esearch = {"db": "pubmed","term": query,"retmode": "json","retstart": 0,"retmax": 10000,'email':'your_email@example.com',
              'datetype':'pdat','mindate':'2010/01/01', 'maxdate':'2025/01/01'}
    response_esearch=requests.get(base_url_esearch, params=params_esearch)
    list_of_pubmed=response_esearch.json()['esearchresult']['idlist']
    dois=[]
    journal=[]
    base_url_efetch='https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
    for list in [list_of_pubmed[i:i+10]for i in range(0,len(list_of_pubmed),10)]:
        query=f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
        param_efetch={'retmode':'xml',"db": "pubmed","id": ','.join(list),"retstart": 0,"retmax": 1000}
        response_efetch=requests.get(base_url_efetch,params=param_efetch)
        tree = ET.fromstring(response_efetch.text)
        base='https://doi.org/'
        for i in tree.findall('PubmedArticle/PubmedData/ArticleIdList/ArticleId'):
            if i.attrib.get('IdType')=='doi':
                dois.append(base+i.text)
        for i in tree.findall('PubmedArticle/MedlineCitation/Article/Journal/Title'):
            journal.append(i.text)
            
    return {k: v for k, v in zip(dois, journal)}
            

In [6]:
query='free full text[filter] AND "AKT signaling pathway"[Title/Abstract] AND review[Publication Type] AND (humans[MeSH Terms] AND ("disease"[MeSH Terms] OR disease[Title/Abstract]))'
dicss=fetch_ncbi_data(query)

In [74]:
all_docs=[]
for i in dicss.keys(): 
    doc=WebBaseLoader(i,requests_kwargs={'allow_redirects': True}).load()[0]
    if len(doc.page_content)>2000:
        all_docs.append(doc)


In [75]:
#doc=WebBaseLoader('https://pmc.ncbi.nlm.nih.gov/articles/PMC11554381/').load()
cleaned_docs=[]
count=0
for doc in all_docs:
    if 'Reference' in doc.page_content and 'Abstract' in doc.page_content:
        references=doc.page_content.split(r'.References')[:-1]
        title=doc.metadata['title'].strip()
        cleaning=re.sub(r"\s+", " ", doc.page_content)
        cleaning=re.split(r"Abstract|background",cleaning)[1]
        cleaning=re.split(r"Acknowledgements",cleaning)[0]
        cleaning=re.sub(r'.Keywords*\n?',' ',cleaning)
        cleaning=re.sub('Open in a new tab','',cleaning)
        cleaning=re.sub(r'[\r\n]+', r'\n', cleaning)
        cleaning=re.sub(r"\s*\(?\s*(?:Figure|Fig\.?)\s*[\d.]+\s*\)?\s*",'',cleaning,flags=re.IGNORECASE).strip()
        cleaning=re.sub(r'\[[\d\s\W]*?\].','',cleaning,flags=re.IGNORECASE)
        cleaning=re.sub(r"\(([^)]*?\s*et\s*al\.\s*[^)]*?)\)",'',cleaning)
        cleaning=re.sub(r'\(\d+(?:,\d+|-?\d+)*\)\.','.',cleaning)
        cleaning=re.sub(r'\([\w\s]+\d{4},?\s*[\d–-]*\)','',cleaning)
        cleaning=re.sub(r"\([rR]eviewed in\)",'',cleaning,flags=re.IGNORECASE)
        cleaning=re.sub(r"www\.[^/]+/",' ',cleaning)
        cleaned_nlp=linker(nlp(cleaning))
        new_text=cleaning
        for i in reversed(cleaned_nlp.ents):
            id=i._.kb_ents
            if id:
                name=linker.kb.cui_to_entity[id[0][0]].canonical_name
                new_text = new_text[:i.start_char] + name + ' ' + i.label_ + new_text[i.end_char:]
        doc.page_content=new_text
        cleaned_docs.append(doc)

In [77]:
splitter = RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=500)
chunks=splitter.split_documents(cleaned_docs)

In [ ]:
embeddings_model = OllamaEmbeddings(model="qwen3-embedding:0.6b",validate_model_on_init=True)
vectorstore=FAISS.from_documents(chunks,embeddings_model)
retriever=vectorstore.as_retriever(search_type='similarity',search_kwargs={"k": 10, "fetch_k": 20})


In [83]:
vectorstore.save_local("/home/bionik/AI_ML/FDA_Project/LLM/faiss_index")

In [9]:
vectorstore = FAISS.load_local(
    "/home/bionik/AI_ML/FDA_Project/LLM/faiss_index",
    embeddings_model,
    allow_dangerous_deserialization=True 
)

In [84]:
class Triplet(BaseModel):
    subject: str = Field(..., description="ONE protein or ONE protein complex")
    predicate: str = Field(..., description="Relationship between protein entities")
    object: str = Field(..., description="ONE protein or ONE protein complex")

class TripletList(BaseModel):
    triplets: List[Triplet]

In [121]:
LLM_model = ChatOllama(model="qwen3:4b-instruct-2507-q4_K_M",validate_model_on_init=True)


In [ ]:
system_template = """
Use the following pieces of retrieved context to answer the question of research article of biology. 
If you don't know the answer, genuinely say that you don't know.
Use only the information which was provided in this query
Keep the answer detailed and precise.

Use this context to answer the question:
{context}
"""

human_message='This is the question: {question}'

prompt = ChatPromptTemplate.from_messages([SystemMessagePromptTemplate.from_template(system_template),
                                           HumanMessagePromptTemplate.from_template(human_message)])

In [116]:
def format_context(retrived_doc):
    context="\n\n".join([doc.page_content for doc in retrived_doc])
    return context


In [117]:
LLM_chain = (
    RunnableParallel({"context": retriever | format_context, "question": RunnablePassthrough()})
    | prompt
    | LLM_model
)

In [118]:
input_data = 'how AKT signaling pathway can be used as therapeutic target'
response = LLM_chain.invoke(input_data)

In [120]:
print(response.content)

The AKT (Protein Kinase B) signaling pathway is a critical intracellular signaling cascade involved in regulating key cellular processes such as cell survival, proliferation, growth, metabolism, and apoptosis. Because of its central role in maintaining cellular homeostasis and its frequent dysregulation in cancer, the AKT pathway has emerged as a promising therapeutic target in oncology and other diseases.

Here’s how the AKT signaling pathway can be used as a therapeutic target:

---

### 1. **In Cancer Therapy: Targeting AKT to Inhibit Tumor Growth and Promote Apoptosis**

**Background:**  
AKT is frequently hyperactivated in various cancers (e.g., breast, prostate, colorectal, lung, and pancreatic cancers) due to mutations, amplifications, or upstream signaling abnormalities (e.g., PI3K activation). This hyperactivation promotes uncontrolled cell growth and resistance to apoptosis.

**Therapeutic Strategies:**

- **AKT Inhibitors**  
  Small-molecule inhibitors that directly block A

In [99]:
LLM_model = ChatOllama(model="qwen3:4b-instruct-2507-q4_K_M",validate_model_on_init=True)
embeddings_model = OllamaEmbeddings(model="qwen3-embedding:0.6b",validate_model_on_init=True)


In [102]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vectorstore.similarity_search(query, k=10)
    serialized = "\n\n".join([doc.page_content for doc in retrieved_docs])
    return serialized, retrieved_docs

In [ ]:
checkpointer = InMemorySaver()
tools = [retrieve_context]
# If desired, specify custom instructions
prompt = ("""
You have access to a tool that retrieves context
Use the tool to help answer user queries.
Use the following pieces of retrieved context to answer the question of research article of biology. 
Keep the answer detailed and precise.""")
agent = create_agent(LLM_model, tools, system_prompt=prompt,checkpointer=checkpointer,
                     middleware=[SummarizationMiddleware(model=LLM_model,trigger=("tokens", 4000),keep=("messages", 10))])

In [124]:
query = ("how AKT signaling pathway can be used as therapeutic target?")
config: RunnableConfig = {"configurable": {"thread_id": "1"}}
response=agent.invoke({"messages": [{"role": "user", "content": query}]},config)

In [127]:
#print(response['messages'][-1].content)
print(response['messages'][-1].content)

The **AKT signaling pathway** (also known as the **PI3K-AKT-mTOR pathway**) is a central regulator of cellular processes such as **cell survival, proliferation, metabolism, and apoptosis**. Due to its critical role in maintaining cellular homeostasis and its frequent dysregulation in diseases—particularly cancer and metabolic disorders—**AKT signaling has emerged as a key therapeutic target**.

Below is a detailed and precise explanation of how the AKT signaling pathway can be used as a therapeutic target, based on current biological and clinical research:

---

### 🔬 1. **Molecular Mechanism of AKT Signaling**

The AKT pathway is activated through a cascade:

1. **Receptor activation** (e.g., growth factor receptors like EGFR, IGF-1R).
2. **Phosphorylation of PI3K** → generates PIP3.
3. **PIP3 recruits AKT** to the cell membrane.
4. **AKT is phosphorylated** at Thr308 and Ser473 by PDK1 and mTORC2, respectively.
5. **Activated AKT** phosphorylates downstream targets (e.g., GSK-3β, mTO

In [128]:
query = ("how this pathway inhibition affect the NFKB(NF-Kappa B) signalling?")
response2=agent.invoke({"messages": [{"role": "user", "content": query}]},config)

In [140]:
response['messages'][2]

ToolMessage(content='and other therapies (most patients received oral sunitinib as second-line treatment). Of the 28 patients in the study who failed prior treatment with imatinib, 23 were evaluable, and 4 of those patients (17.4%) were progression-free at 4 months. In addition, 47 patients enrolled in the trial had failed treatment with first-line imatinib and second-line sunitinib; among the 35 patients who were evaluable, 13 (37.1%) were progression-free at 4 months. Most patients reported AEs: Sixty-seven percent experienced grade 3 or 4 AEs, and 48% experienced SAEs. These results suggest that patients with GIST may benefit from combined treatment in case of first-line and second-line treatment failure. In another phase 2 study, everolimus was studied in patients with STS or bone sarcoma, but limited clinical efficacy was observed (CR/PR or SD rate, 20%). The most common AEs were skin toxicity, mucositis, and fatigue; serious AEs included pneumonitis and anemia.96 Everolimus is be

In [130]:
print(response2['messages'][-1].content)

The **inhibition of the AKT signaling pathway** has significant and well-documented effects on the **NF-κB (Nuclear Factor kappa B) signaling pathway**, primarily through **cross-talk and regulatory interactions** between the two. Understanding this interaction is critical in cancer, inflammation, and immune regulation.

Below is a **detailed, precise, and scientifically grounded explanation** of how AKT inhibition affects NF-κB signaling:

---

## 🔬 1. **Overview of NF-κB Signaling**

NF-κB is a transcription factor that regulates genes involved in:
- Inflammation
- Immune response
- Cell survival
- Proliferation
- Apoptosis

Under resting conditions, NF-κB is sequestered in the cytoplasm by **IκB** (inhibitor of κB). Upon activation (e.g., by TNF-α, IL-1, or TLR ligands), IκB is phosphorylated and degraded, allowing NF-κB to translocate to the nucleus and activate target genes.

---

## 🔬 2. **Cross-Talk Between AKT and NF-κB Pathways**

AKT and NF-κB are **interconnected** through *

In [142]:
ans=LLM_model.invoke('how AKT signaling pathway can be used as therapeutic target?')

In [145]:
print(ans.content)

The AKT (Protein Kinase B) signaling pathway is a central regulator of cell survival, growth, proliferation, metabolism, and apoptosis. Its dysregulation is commonly associated with cancer, diabetes, and other diseases. Because of its critical role in cellular functions, the AKT pathway has emerged as a promising therapeutic target. Here's how AKT signaling can be used as a therapeutic target across various diseases:

---

### 1. **In Cancer Therapy**

AKT is frequently hyperactivated in many cancers (e.g., breast, prostate, colorectal, lung, and pancreatic cancers), often due to upstream mutations (e.g., PI3K mutations, PTEN loss, or growth factor receptor activation).

**Therapeutic Strategies:**

- **AKT Inhibitors**:
  - Small-molecule inhibitors (e.g., **Ipatasertib**, **Capivasertib**, **MK-2206**) directly inhibit AKT kinase activity.
  - These drugs block downstream signaling, leading to reduced cell proliferation and increased apoptosis in cancer cells.
  - Used in combination

In [146]:
retriever.invoke('how AKT signaling pathway can be used as therapeutic target?')

[Document(id='25211740-6305-49b2-a4a8-84c2884f1911', metadata={'source': 'https://doi.org/10.1097/MD.0000000000033593', 'title': '\r\n\tMedicine\r\n', 'language': 'en-US'}, page_content='Bubble chart. BP = biological process, CC = cellular component, GO = Gene Ontology, MF = molecular function.Furthermore, we selected the first 30 KEGG pathways for analysis, and the following KEGG pathways were identified: Lipid and atherosclerosis signaling pathway , human cytomegalovirus infection signaling pathway , PI3K-Akt signaling pathway , Kaposi sarcoma-associated herpesvirus infection signaling pathway , and chemical carcinogenesis receptor activation signaling pathway .: KEGG pathway enrichment analysis of the target proteins. (A) BFAR DNA chart. (B) Bubble chart. KEGG = Kyoto Encyclopedia of Genes and Genomes.3.6. Docking simulation verification AutoDockTools version 1.5.6 was used for molecular docking verification. Molecular docking analysis was used to validate the potential binding of t